# Byte-Pair Encoding tokenization

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

关于BPE算法的文档描述： https://huggingface.co/learn/nlp-course/en/chapter6/5?fw=pt

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

GPT 模型通常使用 UTF-8 编码
GPT 系列（如 GPT-2）通常采用 UTF-8 编码 + BPE 分词，完全支持多语言；
UTF-8 是目前最常用的文本编码，也更适合网络和跨平台应用。
UTF-8 是一种可变长编码，可以编码所有 Unicode 字符，包括：
- 拉丁字母（英文、德语等）
- 汉字（中文）
- 假名（日本语）
- 韩文
- 阿拉伯文
- Emoji 等

它是全球使用最广泛的 Unicode 编码方式，多语言支持完全没有问题。

1. 首字节的高位模式用于表示“总长度”
UTF-8 使用每个字符的首字节前缀来表示这个字符使用了多少个字节进行编码：

字节数	首字节的二进制前缀	可编码范围（十进制）  
1 字节	0xxxxxxx	0～127（ASCII）  
2 字节	110xxxxx	128～2047  
3 字节	1110xxxx	2048～65535  
4 字节	11110xxx	65536～1114111（即 0x10FFFF）  

每个非首字节都以 10xxxxxx 开头，称为 continuation byte。  

首先进行预分词，基本上是按照空格进行切分，空格分配到后续紧跟的单词上  
在 GPT-2 分词器中，Ġ（实际是 Unicode 字符 U+0120，叫作 Latin Capital Letter G with dot above）用来表示 一个空格（或词的边界）。
示例说明：
- 'This' → 没有 Ġ，说明它是句子的起始词，前面没有空格。
- 'Ġis' → 前面有 Ġ，说明原始字符串是 " is"（有一个空格）。
- 'Ġthe' → 对应原始文本 " the"。
- 'ĠHugging' → 对应 " Hugging"。
也就是说，GPT-2 分词器会把空格信息编码进 token 本身，而不是单独作为一个 token 保留。  


下面的代码在分词的基础上进一步统计单词出现的频率

In [ ]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

进一步将单词切分成没有重复的字符集合，且按字符顺序进行排序。

In [ ]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

In [ ]:
# 添加特殊token
vocab = ["<|endoftext|>"] + alphabet.copy()

In [ ]:
# 对原先的每一个单词进行字符切分形成单词作为key和对应的字符列表作为value的词典splits
splits = {word: [c for c in word] for word in word_freqs.keys()}

In [ ]:
# 统计相邻的字符出现的频率，为所有词频字典中unique的单词中相邻的字符对在不同单词中出现（对应出现的次数为freq）的总体统计
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [ ]:
# 将其前面5个字符对打印出来
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

In [ ]:
# 获取频率最高的相邻字符对
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

In [ ]:
# 将最高的相邻字符对进行合并并放入到词汇表中
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

In [ ]:
# 将合并的结果更新到单词及其切分结果的字典里
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [ ]:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

In [ ]:
#合并最高的相邻的现有词汇，并更新到词汇表里，不断持续这个过程支持达到词汇表定义的长度
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [ ]:
print(merges)

In [ ]:
print(vocab)

In [ ]:
# 基于新的词汇表，将文本进行分词的并进行输出
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    # 对于所有的merge生成的新的词汇，更新每个单词的切分结果
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, []) #用是将 splits 中的所有子列表拼接成一个单一的列表

In [ ]:
tokenize("This is not a token.")